In [1]:
#  jojo_trainer.ipynb
#  3.モデルの学習を行う

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import sys
sys.path.append("/content/drive/MyDrive/jojo_poser/src")

import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
!pip install torchinfo
from torchinfo import summary

from image_loader import *
from models import *

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("使用されるデバイス: ",device)

In [5]:
LABELS = ["Buccellati", "Dio", "Giorno", "Highway-Star", "Jo-suke", "Jo-taro",
            "Kakyoin", "Kira", "Kishibe", "Polnareff", "Trish"]

In [ ]:
#  画像の読み込み
im_rows = 256
im_cols = 256

#  Dataを取得
root_path = "/content/drive/MyDrive/jojo_poser"
train_imgs, train_labels = data_loder(root_path,"train", im_rows, im_cols)
valid_imgs, valid_labels = data_loder(root_path,"valid", im_rows, im_cols)

#  Datasetを作成
tr_data = PreprocessJOJO(train_imgs, train_labels, "train")
val_data = PreprocessJOJO(valid_imgs, valid_labels, "valid")
print('訓練データのサイズ: ', tr_data.__len__())
print('検証データのサイズ: ', val_data.__len__())

#  DataLorderを作成
batch_size = 4
tr_batch = data.DataLoader(
    tr_data,                #  訓練用data
    batch_size = batch_size,#  ミニバッチのサイズ
    shuffle = True,         #  シャッフルして抽出
    )
val_batch = data.DataLoader(
    val_data,               #  検証用data
    batch_size = batch_size,#  ミニバッチのサイズ
    shuffle = False,        #  シャッフルはせずに抽出
    )
print('訓練データのミニバッチの個数: ', tr_batch.__len__())
print('検証データのミニバッチの個数: ', val_batch.__len__())

#  DataLoaderをdictにまとめる
dataloaders_dict = {"train":tr_batch, "valid":val_batch}

In [ ]:
#  訓練用のDataLorderをイテレーターに変換
batch_iterator = iter(dataloaders_dict["train"])
#  最初のミニバッチを取り出す
images, labels = next(batch_iterator)
print('ミニバッチのイメージの形状: ',images.size())
print('ミニバッチのラベルの形状: ',len(labels))
print('labels[0]の形状: ',labels[0].size())

In [ ]:
#   モデルのインスタンス作成
net = JOJO_classifier('train', len(LABELS))
#  vggモデルの学習済みの重みを適用
vgg_weights = torch.load(root_path+'/weights/vgg16_reducedfc.pth')
net.vgg.load_state_dict(vgg_weights)
print("[model vgg] weights is applied.")
#  denseモデルの重みを初期化
if isinstance(net.dense, nn.Linear):
    init.kaiming_normal_(net.dense.weight.data)
    if net.dense.bias is not None:
        init.constant_(net.dense.bias, 0.0)

print(net)

In [ ]:
summary(
    net,
    input_size = (batch_size, 3, im_rows, im_cols),
    col_names=["input_size","output_size","num_params"])

In [10]:
#  損失関数及びオプティマイザーの作成
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.dense.parameters(),
                lr = 0.01,
                weight_decay = 0.0005)

In [11]:

def train(net, dataloaders_dict, criterion, optimizer, num_epochs):
    '''
    Parameters:
        net(object): VGG+Dense モデル
        datalorders_dict(dict(object)): DataLorder
        criterion(object): 損失関数
        optimizer(object): オプティマイザー
        num_epochs(int): 学習回数
    '''
    
    net.to(device)
    torch.backends.cudnn.benchmark = True
    
    iteration = 1 #  イテレーション(ステップ)カウンター
    epoch_train_loss = 0.0 #  訓練1エポックごとの損失和
    epoch_val_loss = 0.0 #  検証1エポックごとの損失和
    logs = [] #  損失のログを記録するリスト
    
    for epoch in range(num_epochs):
        print('----------------------------------------------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('----------------------------------------------------')

        for phase in ["train","valid"]:

            if phase=="train":
                #  モデルを訓練モードに
                net.train()
            else:
                if((epoch+1)%10 == 0):
                    net.eval()#  モデルを検証モードに
                    print("----------------------------------------------------")
                    print("----- validation mode -----")
                else:
                    continue
                    
            #  1ステップにおけるミニバッチを使用した学習または検証
            #  データローダーをイテレートしてミニバッチを抽出
            for images, labels in dataloaders_dict[phase]:
                #  画像データにデバイスを割り当てる
                images = images.to(device)
                #  教師データをデバイスを割り当てる
                labels = labels.to(device)

                #  optimizerが保持する勾配を0で初期化
                optimizer.zero_grad()

                #  順伝搬(forward)とバックプロパゲーション(訓練時)
                with torch.set_grad_enabled(phase == 'train'):
                    #  順伝播(forward)
                    outputs = net(images)
                    #  labelの損失平均
                    loss = criterion(outputs, labels)
                    
                    #  訓練時はバックプロパゲーションによるパラメーター更新を行う
                    if phase == 'train':
                        loss.backward()  #  バックプロパゲーション

                        # 勾配が大きすぎると不安定になるので
                        # clipで勾配の上限を2.0に制限する
                        nn.utils.clip_grad_value_(net.parameters(),
                                                  clip_value=2.0)
                        # 勾配降下法の更新式を適用してバイアス、重みを更新
                        optimizer.step()

                        # ミニバッチを10個処理(10ステップ)ごとに損失を出力
                        if (iteration % 10 == 0):
                            #  ステップ, 損失を出力
                            print('step( {} )  loss: {:.4f}'.format(iteration, loss.item()))

                        # エポックの損失をepoch_train_lossに加算する
                        epoch_train_loss += loss.item()
                        # ステップ数を1増やす
                        iteration += 1

                    # 検証モードでは順伝播後の損失の記録のみを行う
                    else:
                        epoch_val_loss += loss.item()
                        
        # epochのphaseごとのlossと正解率
        print('---------------------------------------')
        # 訓練データの損失と検証データの損失を出力
        print('train_loss: {:.4f} - val_loss(Every 10 epochs): {:.4f}'.format(epoch_train_loss, epoch_val_loss))

        # エポックごとに損失をdictオブジェクトに保存
        log_epoch = {'epoch': epoch+1,
                     'train_loss': epoch_train_loss,
                     'val_loss': epoch_val_loss}
        # ログのリストに追加
        logs.append(log_epoch)

        # 訓練時の損失和を0で初期化
        epoch_train_loss = 0.0
        # 検証時の損失和を0で初期化
        epoch_val_loss = 0.0
        
        # 1エポック終了ごとにモデルのパラメーター値を保存
        if ((epoch+1) % 10 == 0):
            torch.save(
                net.state_dict(),
                root_path + '/weights/jojo_weights' + str(epoch+1) + '.pth')
            print('--saved weights--')
    # ログのリストをデータフレームに変換
    df = pd.DataFrame(logs)
    # ログファイルに保存
    df.to_csv(root_path + '/outputs/epoch_loss.csv')

In [ ]:
#  学習
num_epochs = 50
train(net,
      dataloaders_dict,
      criterion,
      optimizer,
      num_epochs=num_epochs)